In [1]:
# system tools
import warnings
import json
import sys
import string
import ast

# data cleaning + analysis tools
import pandas as pd
import datetime as dt
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

#nltk tools
import spacy
import lda #Latent Dirichlet Allocation (create topics)
import gensim
from gensim import corpora, models #for constructing document term matrix
#from stop_words import get_stop_words
from gensim.models import Phrases
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.util import ngrams

#set notebook preferences
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')

%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)

Populating the interactive namespace from numpy and matplotlib


### Import JSON file with city metadata 

This including which cities have published the raw Public Record Requests (PRRs) they receive for analysis. 

In [21]:
json_file = '../data/cities.json'

with open(json_file, 'r') as f:
    md = json.load(f)

###  Create dataframe of PRR data for all relevant cities

This dataframe includes PPR data from the **34 cities** in our sample (52 total cities) that had sufficient raw data with the public record request for analysis. Our sample represents cities that host an online PRR portal for submitting requests.  These data were obtained through a variety of methods including:

1. exporting the full archive of PRRs hosted on the online portal as a csv file  
2. scraping the full history of PRR data from portals which publish previous requests, but do not offer a download option (see [scraping notebook](https://github.com/sunlightpolicy/Sunlight_FOIA/blob/master/src/data/NR_Scrape.ipynb))
3. downloading public records request data that has been published on city’s open data portal  
4. submitting a public record request to obtain the archive of PRR data 

It is worth noting that *specificities of the different city portals influence the substance of the public record requests received*. For example, the city of Clearwater, FL has separate request forms for police records and public records, prompting citizens who submit police record requests to provide the specific case number. In addition, while most of the data released by cities is the raw request submitted by citizens, in a few cases the city released a summary of the submitted request prepared by city staff. For example, the Oklahoma City clerk's office released the summary of the request and the department the request was routed to for response. 


In [23]:
data_raw = pd.DataFrame(columns = ['city', 'month_year', 'Summary'])
city_list = []
for key, value in md.items():
    city = value['name']
    filepath = '/Users/alenastern/Google Drive File Stream/My Drive/Alena_Project/PR_Data/{}.csv'.format(city)
    # tag in metadata for whether city publishes request content
    if value["desc"] == "Y":
        try:
            df = pd.read_csv(filepath)
        except:
            try:
                df = pd.read_csv(filepath, encoding='mac_roman')
            except:
                continue
        print(key)
        name = key.split(' ')
        city_list.extend([x for x in name[:-1]])
    else:
        continue
    
    try:
        df['Create Date'] = pd.to_datetime(df['Create Date'])
    except:
        df['New'] = pd.to_datetime(df['Create Date'].apply(lambda x: re.findall('^\S*', x)[0]))
        df.drop(columns=['Create Date'], inplace = True)
        df.rename(index=str, columns={"New": "Create Date"}, inplace = True)

    df['month_year'] = df['Create Date'].dt.to_period('M')
    
    mc = df[['month_year', 'Summary']]
    mc['city'] = city
    
    data_raw = pd.concat([data_raw, mc])


Arlington city
Bainbridge Island city
Boulder County
Cathedral City city
Clearwater city
Dayton city
Denton city
Everett city
Fort Collins city
Greensboro city
Hayward city
Kirkland city
Las Cruces city
Lynnwood city
Mercer Island city
Miami city
Middleborough town
New Orleans city
Oakland city
Oklahoma City city
Olympia city
Palo Alto city
Peoria city
Pullman city
Rancho Cucamonga city
Redmond city
Renton city
Sacramento city
San Francisco city
Tukwila city
Vallejo city
West Sacramento city
Winchester city


#### Our raw dataset includes 110,063 PRRs from 33 different cities

NEED TO EDIT BEFORE FINAL

In [24]:
data_raw.to_csv('data_raw.csv')
data_raw.shape

(110063, 3)

In [25]:
len(data_raw.city.unique())

33

In [26]:
# create sequential numeric index for data

data_raw.index = pd.RangeIndex(len(data_raw.index))
data_raw.reset_index(inplace=True)

### Create dataframe for cleaning by removing null summaries

In [27]:
# Drop observations that are null for the raw PRR content field ('Summary')
data = data_raw.dropna(subset=['Summary'])

We see the raw data below. As we can see, the text in the Summary field is very messy and will require a lot of cleaning to prepare the data for analysis!

In [28]:
data.head()

,index,Summary,city,month_year
0,0,"We are working with an engineering firm on an upcoming project. They have asked us to gather maps for this project. Would you be able to assist me in gathering maps/records (as builds) for any underground water facilities you may have? Something just showing the route of the water lines would do.\n\n207th ST NE to 92nd Ave NE, Arlington, Cascade Surveying & Engineering \n\nI have attached the scope for your convenience. Please let me know if you have questions.",Arlington,2018-06
1,1,"Need copies of contracts and all related documents pertaining to Topcub Aircraft property located at 17922 59th DR NE Arlington WA 98223 between Arlington Airport, Topcub Aircraft, City of Arlington, HCI Steel Buildings and PUD.",Arlington,2018-06
2,2,"Copies of Building Permits of $5,000 valuation and up ($20,000 min for Re-Roofs), ($50,000 min. for Cell Tower upgrades), (Electrical, Mechanical & Plumbing at $100,000 min.) and (Solar Panels, Swimming Pools & Foundations at any valuation)",Arlington,2018-06
3,3,"police report filed to an officer against Wayne Parris (DOB 08-03-1957) from Brittany J. Parris. The paperwork I have has a case number D18-39 it is also stamped at the bottom with 18-1294, Iím not sure which number you will need. If there is any other information needed please let me know.",Arlington,2018-06
4,4,"Email Communications between Stephanie Shook, Dave Kraski, Bruce Stedman and Chad Schmidt in regards to Fire Protection District 21 billing and passage of contract for ALS Services. \n\nAlso any copies of Agenda Bills, D21 Contract and materials presented for review in Nov/Dec time frame in regards to the contract.",Arlington,2018-06


#### Function to convert nltk part of speech tags to wordnet tags (we use this to stem the words in data cleaning below):

In [29]:
def get_wordnet_pos(tag):

    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

#### Function to turn separate files of the 1000 most popular baby names by year provided by the [Social Security Administration](https://www.ssa.gov/OACT/babynames/) into a single set of unique first names across years


In [32]:
def name_list(st, end):
    names = set()
    for yr in range(st, end+1):
        fp = '../data/names/yob{}.txt'.format(yr)
        df = pd.read_table(fp, sep = ',', names = ['name', 'sex', 'count'])
        names |= set(df['name'])
    
    return list(names)

In [33]:
# create list of unique first names that were on the 1000 most popular names each year between 1950-2017

names = name_list(1950,2017)

#### Create list on common surnames in the United States. Data on surnames is from the U.S. Census Bureau, compiled by FiveThirtyEight and accessed via [data.world](https://data.world/fivethirtyeight/most-common-name/workspace/file?filename=README.md)

In [34]:
last_names = pd.read_csv('../data/names/surnames.csv')
last_names.name = last_names.name.str.title()
ln = list(last_names['name'])

In [35]:
# combine first names and surnames and create dictionary 

all_names = names + ln
all_names_dict = {key: 1 for key in all_names if key}

## Clean PRR data to prepare for LDA analysis

Prior to analysis, we clean our unstructured text data to improve the outcome of our LDA analysis results. Our goals are as follows:

1. Remove "noise" - words that do not provide information on the subject of a PRR (eg. stop words like "the", proper nouns like people's names or city names, punctuation and digits, and general words/phrase common to PRRs like "good morning" or "record"
2. Stem words so like words are treated as the same (eg. "photo" and "photos" should be regarded as the same word, as should "assault" and "assaulted"
3. Account for meaningful phrases where the combination of words has particular meaning (to avoid excessive computation time, we only consider two-word phrases)

In [39]:
# make a copy of the original request column to preserve
data['summary_raw'] = data['Summary']

# remove common public record request phrases - we remove as phrases because we care about specific combination/order 
# of words (we want to remove "open record request" not all instances of word "open")
phrase_list = ['public record request', 'open record request', 'public records request', 'open records request', 
               'foia request', 'see attached', 'see attachment', 'to whom it may concern', 'public records act',
              'electronic copy', 'electronic copies', 'freedom of information act', 'good afternoon', 'good morning',
              'good day']
                         
for phrase in phrase_list:
    s = re.compile(re.escape(phrase), re.IGNORECASE)
    data.Summary = data['Summary'].apply(lambda x: s.sub('', x))
    
    
# Replace common acronyms in Summary
data.Summary = data.Summary.str.replace('NOPD', 'police department')
data.Summary = data.Summary.str.replace('OPD' , 'police department')
data.Summary = data.Summary.str.replace('SFPD', 'police department')
data.Summary = data.Summary.str.replace('CPD', 'police department')
data.Summary = data.Summary.str.replace('APD', 'police department')
data.Summary = data.Summary.str.replace('GPD', 'police department')
data.Summary = data.Summary.str.replace('KPD', 'police department')
data.Summary = data.Summary.str.replace('TPD', 'police department')
data.Summary = data.Summary.str.replace('DPD', 'police department')
data.Summary = data.Summary.str.replace('EPD', 'police department')
data.Summary = data.Summary.str.replace('HPD', 'police department')
data.Summary = data.Summary.str.replace('LPD', 'police department')
data.Summary = data.Summary.str.replace('MDPD', 'police department')
data.Summary = data.Summary.str.replace('PPD', 'police department')
data.Summary = data.Summary.str.replace('SPD', 'police department')
data.Summary = data.Summary.str.replace('VPD', 'police department')
data.Summary = data.Summary.str.replace('CCPD', 'police department')
data.Summary = data.Summary.str.replace('FCPD', 'police department')
data.Summary = data.Summary.str.replace('TPD', 'police department')
data.Summary = data.Summary.str.replace('LCPD', 'police department')
data.Summary = data.Summary.str.replace('OKCPD', 'police department')
data.Summary = data.Summary.str.replace('PAPD', 'police department')
data.Summary = data.Summary.str.replace('RCPD', 'police department')
data.Summary = data.Summary.str.replace('WSPD', 'police department')

# PDRD = portable digital recording device (body cam) worn by police
data.Summary = data.Summary.str.replace('PDRD', 'police body camera')
data.Summary = data.Summary.str.replace('CPS', 'child protective services')

#https://www.sfdph.org/dph/EH/HMUPA/HMUPAFormsMenu.asp - hazardous materials
#https://www.waterboards.ca.gov/ust/contacts/docs/lop_guide.pdf - water resources local oversight program
data.Summary = data.Summary.str.replace('LOP', 'water')
data.Summary = data.Summary.str.replace('HMUPA', 'hazardous materials')

# Replace key numbers with strings
data.Summary = data.Summary.str.replace(' 911 ', ' nineoneone ')
data.Summary = data.Summary.str.replace(' 311 ', ' threeoneone ')
data.Summary = data.Summary.str.replace(' 9-11 ', ' nineoneone ')
data.Summary = data.Summary.str.replace(' 3-11 ', ' threeoneone ')

# Remove digits
dig_translator = str.maketrans('','', string.digits)
data.Summary = data.Summary.str.translate(dig_translator)

# because "will" is in the NLTK list of stopwords below, we treat 'final will' separately                         
c = re.compile(re.escape('final will'), re.IGNORECASE)
data.Summary = data['Summary'].apply(lambda x: s.sub('final final_will', x))

# replace hyphen and slash with space to treat hyphate words as two separate words
hyphen_translator = str.maketrans('-/','  ')
data.Summary = data.Summary.str.translate(hyphen_translator)

# remove all punctuation
translator = str.maketrans('','', string.punctuation)
data.Summary = data.Summary.str.translate(translator)

## Use SpaCy tokenizer to ID Proper Nouns ##

#nlp = spacy.load('en_core_web_sm')
#data['token_sp'] = data['Summary'].apply(lambda x: nlp(x))
#data['pn'] = data['token_sp'].apply(lambda x: [ i.lemma_ for i in x if i.tag_ == 'NNP'])

# split text into list of words by space 
data['token'] = data['Summary'].apply(lambda x: nltk.word_tokenize(x))

# remove proper first and last names in our dictionary + convert all words to lower case
data['token'] = data['token'].apply(lambda x: [i.lower() for i in x if i not in all_names_dict])

#remove empty strings, stopwords and stem
stop_words = set(stopwords.words('english'))
lmtzr = WordNetLemmatizer()
data['lemma'] = data['token'].apply(lambda x: nltk.pos_tag(x))
data['mash'] = data['lemma'].apply(lambda x: [lmtzr.lemmatize(i[0], get_wordnet_pos(i[1])) for i in x if len(i[0]) > 0 and i[0] not in stop_words])

# Remove whitespace
wsp_translator = str.maketrans('','', string.whitespace)
data['mash'] = data['mash'].apply(lambda x: [i.translate(wsp_translator) for i in x])

# Remove empty lists
data['mash_len'] = data['mash'].apply(lambda x: len(x))
data = data[data['mash_len'] > 0]


In [40]:
# remove number suffixes
suffix_list = ['th', 'nd', 'st', 'rd', 'blvd', 'pkwy']
data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in suffix_list])

# remove city and state abbreviations
abbv_list = ['wa', 'nc', 'co', 'ca', 'oh', 'tx', 'nm', 'fl', 'ma', 'la', 'ok', 'az', 'ri', 'va', 
             'francisco', 'sf', 'okc', 'lv', 'nola', 'slc', 'cw']
data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in abbv_list])

# remove spelled numbers
num_list = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']
data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in  num_list])

# replace 'inc' with 'incident
data['mash'] = data['mash'].apply(lambda x: ['incident' if i=='inc' else i for i in x])

# replace 'pd' with 'police department
data['mash'] = data['mash'].apply(lambda x: ['police department' if i=='pd' else i for i in x])

# remove noise words
noise = ['dr', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sept', 'sep', 'oct', 'nov', 'dec', 
        'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 
        'december', 'ne', 'nw', 'se', 'sw', 'ct', 'dr', 'way', 'dv', 'ave', 'aka', 'get', 'look', 'im', 'want', 
        'find', 'could', 'go', 'take', 'e', 'n', 's', 'w', '“', '’', '”', '•', 'northeast', 'northwest', 'southeast', 
        'southwest', 'north', 'south', 'east', 'west', 'orleans', '–', 'a', 'b', 'c', 'd', 'f', 'g', 'h', 'i', 'j', 'k',
        'l', 'm', 'o', 'p', 'q', 'r', 't', 'u', 'v', 'x', 'y', 'z', 'am', 'pm', 'hr', 'mr', 'ms', 'mrs', 'johnson', 
        'jr', 'kent', 'christopher', 'miller', 'joe', 'willows', 'david', 'michael', 'john', 'red', 'robert',
        'ask', 'able', 'let', 'question', 'also', 'snohomish', '¬ß', 'per', 'available', 'test', '√Ø', 'andor', '·', 'etc',
        'ï', 'ce', 'eg', 'sammamish']

data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in noise])

# remove cities
city_list = ['arlington', 'asheville', 'bainbridge', 'island', 'boulder', 'cathedral' ,'clearwater', 'dayton', 
            'denton', 'everett', 'fort', 'collins', 'greensboro', 'hayward', 'kirkland', 'las', 'cruces', 'lynnwood',
            'mercer', 'miami', 'middleborough', 'new', 'orleans', 'oakland', 'oklahoma', 'olympia', 'palo', 'alto', 
            'peoria', 'pullman', 'rancho', 'cucamonga', 'redmond', 'renton', 'sacramento', 'san', 'francisco', 
            'tukwila', 'vallejo', 'west', 'sacramento', 'winchester']

data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in city_list])

# Remove all state names
state_list = ['washington', 'carolina', 'colorado', 'california',
             'ohio', 'texas', 'florida', 'new', 'mexico','massachusetts',
             'louisiana', 'oklahoma', 'arizona', 'rhode', 'virginia']

data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in state_list])

# Create two-word phrases (bigrams)
data['bigrams'] = data['mash'].apply(lambda x: ["_".join(w) for w in ngrams(x, 2)])

#### Identify and remove noise words that are commonly used in PRRs

In [41]:
word_list = [y for x in list(data['mash']) for y in x]
counts = Counter(word_list)
Counter(word_list).most_common(50)

[('report', 43899),
 ('request', 31182),
 ('record', 23179),
 ('please', 21970),
 ('copy', 20125),
 ('property', 13058),
 ('information', 12020),
 ('number', 11451),
 ('provide', 10819),
 ('would', 10607),
 ('include', 10494),
 ('incident', 10264),
 ('document', 9837),
 ('police', 9587),
 ('permit', 9376),
 ('accident', 9038),
 ('thank', 8925),
 ('address', 8436),
 ('department', 8347),
 ('like', 8025),
 ('location', 7985),
 ('regard', 7839),
 ('date', 7570),
 ('email', 7463),
 ('relate', 6769),
 ('reference', 6760),
 ('building', 6719),
 ('insurance', 6603),
 ('call', 6573),
 ('case', 6549),
 ('following', 6439),
 ('video', 6264),
 ('type', 5770),
 ('violation', 5628),
 ('file', 5622),
 ('plan', 5469),
 ('auto', 5393),
 ('need', 5212),
 ('dob', 4978),
 ('name', 4857),
 ('insure', 4774),
 ('public', 4498),
 ('transaction', 4476),
 ('city', 4416),
 ('make', 4366),
 ('occurrence', 4360),
 ('project', 4336),
 ('list', 4318),
 ('involve', 4190),
 ('time', 4119)]

We see that general words like "report", "please", and "copy" ae very common, yet do not provide information on the content of the request. We remove these common and general words, keeping common words that provide information like "property" and "police".

In [42]:
common_list = ['report', 'request', 'record', 'city', 'please', 'copy', 'date', 'information', 'would', 'regard', 'public',
              'include', 'document', 'provide', 'like', 'thank', 'need', 'know', 'thanks', 'pursuant', 'dear', 'file',
              'relate', 'from', 'either', 'hello', 'hi', 'foia', 'requestors', 'requestor', 'receive', 'available', 
               'make', 'attach', 'pertain', 'might', 'see', 'near']

# remove general words that are common to public record requests
data['mash'] = data['mash'].apply(lambda x: [i for i in x if i not in common_list])

#### Identify meaningful phrases by looking at the list of two-word sequences (bigrams) that are frequently used in public record requests. The meaningful phrases that we identify will be added to the list of words to consider in analysis for the PRRs in which they appear.

In [43]:
bigram_list = [y for x in list(data['bigrams']) for y in x]
counts = Counter(bigram_list)
Counter(bigram_list).most_common(100)

[('would_like', 7167),
 ('please_provide', 6801),
 ('accident_report', 5311),
 ('police_report', 4861),
 ('report_number', 4851),
 ('report_type', 4341),
 ('transaction_reference', 4315),
 ('auto_accident', 4299),
 ('occurrence_location', 4227),
 ('reference_insurance', 4207),
 ('number_occurrence', 4036),
 ('type_auto', 3557),
 ('request_copy', 3259),
 ('incident_report', 2889),
 ('offense_report', 2493),
 ('like_request', 2180),
 ('code_violation', 2174),
 ('provide_copy', 2092),
 ('police_department', 1894),
 ('include_limited', 1732),
 ('public_record', 1705),
 ('property_locate', 1698),
 ('certificate_occupancy', 1693),
 ('copy_police', 1621),
 ('hazardous_material', 1609),
 ('request_record', 1563),
 ('building_permit', 1485),
 ('storage_tank', 1410),
 ('please_know', 1288),
 ('request_please', 1274),
 ('please_advise', 1223),
 ('please_send', 1198),
 ('document_relate', 1183),
 ('please_contact', 1164),
 ('law_enforcement', 1147),
 ('copy_report', 1102),
 ('collision_report', 10

While some of the common bigrams do not provide insight (such as "would_like" and "please_provide") others provide key meaning that is not captured by the individual words such as "police_report", "auto_accident", and "building_permit". We add the meaningful bigrams in the list below to the final set of words for the PRRs in which they occur. 

In [44]:

common_bigrams = ['police_report', 'insurance_company', 'location_loss', 'date_occurrence', 'reportcase_number',
                  'insure_driver', 'auto_accident', 'occurrence_location', 'transactionreference_insurance', 'number_date', 'type_auto',
                  'accident_reportcase', 'code_violation', 'copy_police', 'incident_report', 'police_department', 'certificate_occupancy',
                  'accident_report', 'property_locate', 'storage_tank','driver_note', 'building_permit', 'driver_driver','case_number', 
                  'hazardous_material', 'collision_report', 'state_farm', 'site_plan', 'fire_department', 'ftp_report', 'auto_theft',
                  'fire_code', 'request_police', 'farm_claim', 'claim_compass', 'site_assessment', 'compass_report', 'environmental_site', 
                  'tax_sale', 'loss_cross','city_council', 'code_enforcement', 'subject_property', 'report_case', 'phase_environmental', 
                  'report_incident', 'date_loss', 'police_case', 'witness_statement', 'driving_record', 'break_in', 'birth_certificate', 
                  'death_certificate', 'background_check', 'public_works', 'lease_agreement', 'medical_record', 'billing_record', 
                  'record_check', 'records_check', 'marriage_certificate', 'marriage_record', 'park_ticket', 'miss_person',
                 'marriage_license', 'reckless_driving', 'arrest_report', 'medical_billing', 'medical_report', 'criminal_record',
                 'floor_plan', 'site_plan', 'building_plan', 'building_code', 'code_enforcement', 'personnel_file']

data['common_bigrams'] = data['bigrams'].apply(lambda x: [i for i in x if i in common_bigrams])

#### Combine columns containing cleaned words (mash) and meaningful phrases (common_bigrams) to yield final set of words for analysis for each PRR

In [45]:
data['final_mash'] = data['mash'] + data['common_bigrams']

# Remove empty lists
data['mash_len'] = data['final_mash'].apply(lambda x: len(x))
data = data[data['mash_len'] > 0]

#### We can see the result of the final cleaned data below. 
The final_mash column represents the set of words that will be considered in our analysis.

In [46]:
data.head()

,index,Summary,city,month_year,token,lemma,mash,mash_len,bigrams,common_bigrams,final_mash
0,0,We are working with an engineering firm on an upcoming project They have asked us to gather maps for this project Would you be able to assist me in gathering maps records as builds for any underground water facilities you may have Something just showing the route of the water lines would do\n\nth ST NE to nd Ave NE Arlington Cascade Surveying Engineering \n\nI have attached the scope for your convenience Please let me know if you have questions,Arlington,2018-06,"[are, working, with, an, engineering, firm, on, an, upcoming, project, they, have, asked, us, to, gather, maps, for, this, project, would, you, be, able, to, assist, me, in, gathering, maps, records, as, builds, for, any, underground, water, facilities, you, may, have, something, just, showing, the, route, of, the, water, lines, would, do, th, st, ne, to, nd, ne, cascade, surveying, engineering, i, have, attached, the, scope, for, your, convenience, please, let, me, know, if, you, have, questions]","[(are, VBP), (working, VBG), (with, IN), (an, DT), (engineering, NN), (firm, NN), (on, IN), (an, DT), (upcoming, JJ), (project, NN), (they, PRP), (have, VBP), (asked, VBN), (us, PRP), (to, TO), (gather, VB), (maps, NNS), (for, IN), (this, DT), (project, NN), (would, MD), (you, PRP), (be, VB), (able, JJ), (to, TO), (assist, VB), (me, PRP), (in, IN), (gathering, VBG), (maps, NNS), (records, NNS), (as, IN), (builds, NNS), (for, IN), (any, DT), (underground, JJ), (water, NN), (facilities, NNS), (you, PRP), (may, MD), (have, VB), (something, NN), (just, RB), (showing, VBG), (the, DT), (route, NN), (of, IN), (the, DT), (water, NN), (lines, NNS), (would, MD), (do, VB), (th, VB), (st, VB), (ne, JJ), (to, TO), (nd, VB), (ne, JJ), (cascade, NN), (surveying, VBG), (engineering, NN), (i, NN), (have, VBP), (attached, VBN), (the, DT), (scope, NN), (for, IN), (your, PRP$), (convenience, NN), (please, NN), (let, VB), (me, PRP), (know, VB), (if, IN), (you, PRP), (have, VBP), (questions, NNS)]","[work, engineering, firm, upcoming, project, gather, map, project, assist, gather, map, build, underground, water, facility, something, show, route, water, line, cascade, survey, engineering, scope, convenience]",25,"[work_engineering, engineering_firm, firm_upcoming, upcoming_project, project_gather, gather_map, map_project, project_would, would_assist, assist_gather, gather_map, map_record, record_build, build_underground, underground_water, water_facility, facility_something, something_show, show_route, route_water, water_line, line_would, would_cascade, cascade_survey, survey_engineering, engineering_attach, attach_scope, scope_convenience, convenience_please, please_know]",[],"[work, engineering, firm, upcoming, project, gather, map, project, assist, gather, map, build, underground, water, facility, something, show, route, water, line, cascade, survey, engineering, scope, convenience]"
1,1,Need copies of contracts and all related documents pertaining to Topcub Aircraft property located at th DR NE Arlington WA between Arlington Airport Topcub Aircraft City of Arlington HCI Steel Buildings and PUD,Arlington,2018-06,"[copies, of, contracts, and, all, related, documents, pertaining, to, topcub, aircraft, property, located, at, th, dr, ne, wa, between, airport, topcub, aircraft, of, hci, buildings, and, pud]","[(copies, NNS), (of, IN), (contracts, NNS), (and, CC), (all, DT), (related, JJ), (documents, NNS), (pertaining, VBG), (to, TO), (topcub, VB), (aircraft, NN), (property, NN), (located, VBN), (at, IN), (th, NN), (dr, NN), (ne, JJ), (wa, NN), (between, IN), (airport, NN), (topcub, NN), (aircraft, NN), (of, IN), (hci, NN), (buildings, NNS), (and, CC), (pud, NN)]","[contract, related, topcub, aircraft, property, locate, airport, topcub, aircraft, hci, building, pud]",13,"[copy_contract, contract_related, related_document, document_pertain, pertain_topcub, topcub_aircraft, aircraft_property, property_loca

After cleaning, we have 89,145 observations (this means that just over 10,000 PRRs contained no information that could be used for analysis. For example, a PRR that is just a person's name or an alphanumeric case number). We see below that the average PRR has just over 13 words in the final mash. It is worth noting that there are a significant number of PRRs that yield a final mash of only 1 word. As we test various LDA models, we will test a number of different additional criteria to exclude records with limited information from analysis to determine whether the performance of our models improves.

In [47]:
data['mash_len'].describe()

count    89145.000000
mean        13.144114
std         24.484836
min          1.000000
25%          3.000000
50%          6.000000
75%         15.000000
max       2512.000000
Name: mash_len, dtype: float64

#### We can see that there is significant variation in the average request length per city. 
In some cases, cities with short average length represent cities where the provided data represented a summary of the original request (Oklahoma City) though in other cases, like Dayton, we received the raw data and the average length is still considerably shorter than other cities. 

In [48]:
data_gp = data.groupby('city').mean()
data_gp['mash_len']

city
Arlington          11.429670
Bainbridge         17.887482
Boulder            23.303030
CathedralCity      12.507463
Clearwater          7.233235
Dayton              2.617308
Denton             20.767620
Everett            13.735452
FortCollins        64.732143
Greensboro          2.329954
Hayward            32.089041
Kirkland           10.635076
LasCruces          26.474926
Lynnwood           16.546875
Mercer             16.500000
Miami              31.863489
Middleborough      26.000000
Nola               27.552862
OKC                 3.289256
Oakland            16.951460
Olympia            11.122305
PaloAlto           31.666667
Peoria              7.336830
Pullman             8.857691
RanchoCucamonga    22.843750
Redmond            12.497893
Renton             18.460102
Sacramento          4.783000
SanFrancisco       25.152969
Tukwila             6.114954
Vallejo            35.956268
WestSacramento     21.626126
Winchester         23.638418
Name: mash_len, dtype: float64

#### We can see a couple of examples of the cleaned mash and the original request:

In [49]:
data['Summary'][data.index == 164]

164    Police Traffic Collision Report and any other police reports records or documents relating to \nCase     \n\nUnit    Carla Jaramillo\nUnit    Jordan Boss\n\nTime of incident   
Name: Summary, dtype: object

In [50]:
data['final_mash'][data.index == 164]

164    [traffic, collision, police, unit, unit, time, incident, collision_report, police_report]
Name: final_mash, dtype: object

In [51]:
data['final_mash'][data.index == 60000]

60000    [display, bidder, individual, line, item, bid, price, youth, job, award, bid, proposal]
Name: final_mash, dtype: object

In [52]:
data["Summary"][60000]

'I would like a copy of\xa0the records displaying the bidders and their individual line item bid prices for the Youth Study Center Streets job that was awarded bid proposal no C  Thank you'

#### Save the cleaned data to csv to use for testing different LDA models

In [53]:
data.to_csv('data.csv', index=False)

Next, we tested a number of different parameters for the LDA models to identify the optimal model. Because these models are very computationally intensive and take a long time to run, we have included the tests and final model in a [separate notebook](https://github.com/sunlightpolicy/Sunlight_FOIA/blob/master/src/analysis/LDA_Model_Tests.ipynb). Below, we conduct the analysis on our final model.

# LDA Analysis

In [ ]:
# Test different length restrictions
'''
# 1) as-is

# create dictionary and corpus
texts = list(data['final_mash'])
dictionary = corpora.Dictionary(texts)
corpus_all = [dictionary.doc2bow(text) for text in texts]

lda_60_60 = gensim.models.ldamodel.LdaModel(corpus_all, num_topics=60, id2word = dictionary, 
                                         passes = 60, random_state=7)
model_name = "lda_60_60_model_all"
lda_60_60.save(model_name)
corpus_lda = lda_60_60[corpus_all]
corpus_lda_list = list(corpus_lda)
topics = data.copy()
topics = topics.assign(topic_comp = corpus_lda_list)
file_name = "topics/lda_60_60_topics_all.csv"
topics.to_csv(file_name)
'''

# 2) mash len > 2

data_ml2 = data[data['mash_len'] > 2]
texts = list(data_ml2['final_mash'])
dictionary = corpora.Dictionary(texts)
corpus_ml2 = [dictionary.doc2bow(text) for text in texts]

lda_100_60_ml2 = gensim.models.ldamodel.LdaModel(corpus_ml2, num_topics=100, id2word = dictionary, 
                                         passes = 60, random_state=7)
model_name = "lda_100_60_model_ml2"
lda_100_60_ml2.save(model_name)
corpus_lda = lda_100_60_ml2[corpus_ml2]
corpus_lda_list = list(corpus_lda)
topics = data_ml2.copy()
topics = topics.assign(topic_comp = corpus_lda_list)
file_name = "topics/lda_100_60_topics_ml2.csv"
topics.to_csv(file_name)

# 3) mash len > 3

data_ml3 = data[data['mash_len'] > 3]
texts = list(data_ml3['final_mash'])
dictionary = corpora.Dictionary(texts)
corpus_ml3 = [dictionary.doc2bow(text) for text in texts]

lda_100_60_ml3 = gensim.models.ldamodel.LdaModel(corpus_ml3, num_topics=100, id2word = dictionary, 
                                         passes = 60, random_state=7)
model_name = "lda_100_60_model_ml3"
lda_100_60_ml3.save(model_name)
corpus_lda = lda_100_60_ml3[corpus_ml3]
corpus_lda_list = list(corpus_lda)
topics = data_ml3.copy()
topics = topics.assign(topic_comp = corpus_lda_list)
file_name = "topics/lda_100_60_topics_ml3.csv"
topics.to_csv(file_name)

In [ ]:
data_ml = data[data['mash_len'] > 2]

In [ ]:
# create dictionary and corpus
texts = list(data['final_mash'])
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
# 30 topics and 30 passes
lda_30_45 = gensim.models.ldamodel.LdaModel(corpus, num_topics=30, id2word = dictionary, 
                                         passes = 30, random_state=7)

In [ ]:
# show topics for model
lda_30_45.show_topics(num_topics=30, formatted=False)

In [ ]:
# show topics for model
lda_50_45_2.show_topics(num_topics=50, formatted=False)

In [ ]:
# save good model
lda_40_45.save('lda_40_45_model')

In [ ]:
lda_40_45 = gensim.models.ldamodel.LdaModel.load('lda_40_45_model')

In [ ]:
corpus_lda = lda_40_45[corpus] #this is just a wrapper; calculates on the fly when you call it



In [ ]:
corpus_lda_list = list(corpus_lda)

In [ ]:
for index, score in sorted(lda_30_45[corpus[600]], key=lambda tup: -1*tup[1]): #600th document
    print("Score: {}\t Topic: {} \n".format(score, lda_30_45.print_topic(index, 15))) #15 word topics

In [ ]:
data.mash2[600]

In [ ]:
topics = data.copy()
topics = topics.assign(topic_comp = corpus_lda_list)
topics.head() #the topic_comp column are actual Python lists

In [10]:
def highest_topic(fp):
    df = pd.read_csv(fp)
    df['topic_comp'] =  df['topic_comp'].apply(lambda x:  ast.literal_eval(x))
    df['comp_len'] = df['topic_comp'].apply(len)
    df = df[df['comp_len'] > 0]
    df['top_topic'] = df['topic_comp'].apply(lambda x: max(x, key=lambda item:item[1])[0])
    df['top_topic_comp'] = df['topic_comp'].apply(lambda x: max(x, key=lambda item:item[1])[1])
    
    return df
    
    

In [8]:
def topics_to_csv(df, num_topics):
    for topic in range(0, num_topics):
        subset = df[df['top_topic'] == topic]
        file_name = 'topics/{}_PRR_topic_{}.csv'.format(num_topics, topic)
        subset.to_csv(file_name)

In [ ]:
def process_csv(model_list):
    for model in model_list:
        fp = 'topics/lda_{}_45_topics.csv'.format(model)
        df = pd.read_csv(fp)
        highest_topic(df, fp)
        topics_to_csv(df, model)
    

In [ ]:
fp = 'topics/lda_data_c2000_3.csv'
highest_topic(fp)
topics_to_csv(fp, 60)

In [ ]:
fp2 = 'topics/lda_data10.csv'
highest_topic(fp2)
topics_to_csv(fp2, 60)

In [ ]:
df.head()

In [ ]:
d = highest_topic('topics/lda_data_avg.csv')

In [ ]:
d.head()

In [ ]:
df = df[df['comp_len'] > 0]

In [ ]:
df_blank = df[df['comp_len'] == 0 ]

In [ ]:
df_blank

#### We test a variety of different numbers of topics to identify the number of topics that yields the best results:

In [ ]:
num_topics_list = [20, 30, 40, 50, 60]
for n in num_topics_list:
    lda = gensim.models.ldamodel.LdaModel(corpus, num_topics=n, id2word = dictionary, 
                                         passes = 60, random_state=7)
    
    model_name = "lda_{}_45_model".format(n)
    lda.save(model_name)
    corpus_lda = lda[corpus]
    corpus_lda_list = list(corpus_lda)
    topics = data.copy()
    topics = topics.assign(topic_comp = corpus_lda_list)
    file_name = "topics/lda_{}_45_topics.csv".format(n)
    topics.to_csv(file_name)
    

In [83]:
data = pd.read_csv('topics/lda_data_avg_2.csv')

In [85]:
dt = highest_topic('topics/lda_data_avg_2.csv')

In [86]:
dt = dt[dt['city'] == 'Greensboro']

In [87]:
dt.to_csv('gb_raw.csv')

In [ ]:
process_csv(num_topics_list)

In [ ]:
data['topic_comp'][500]

In [ ]:
# create small dataset for testing functions

In [ ]:
data_sm = data[:50]

In [ ]:
highest_topic(data)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data2 = data[~data['top_topic'].isna()]

In [ ]:
data2.shape

In [ ]:
data['top_topic'].hist(bins=50)
plt.title('Count by Topic')
plt.show()

In [ ]:
lda_50_45 = gensim.models.ldamodel.LdaModel.load('lda_50_45_model')

In [ ]:
topics_to_csv(data, 50)

## Evaluate Categories

In [ ]:
def eval_cat(df, num_topics, city):
    df_raw = pd.read_csv('/Users/alenastern/Google Drive File Stream/My Drive/Alena_Project/PR_Data/{}.csv'.format(city))
    
    df_raw_gp = df_raw.groupby('Dept').count()
    df_raw_gp['Summary'].plot(kind = 'bar')
    title = 'PRR Departments for {}'.format(city)
    plt.title(title)
    plt.show()
    
    df_city = df[df['city'] == city]
    df_merge = df_city.merge(df_raw, how ='left', on = 'Summary')
    df_merge['top_topic'] = pd.to_numeric(df_merge['top_topic'])
    for topic in range(0, num_topics):
        df_sub = df_merge[df_merge['top_topic'] == topic]
        df_gp = df_sub.groupby('Dept').count()
        df_gp['Summary'].plot(kind='bar')
        title = 'Departments for topic {} in {}'.format(topic, city)
        plt.title(title)
        plt.show()

In [ ]:
city_list = ['Bainbridge', 'LasCruces', 'FortCollins', 'Mercer', 'Nola', 'Oakland', 'PaloAlto', 'Redmond', 
             'SanFrancisco', 'Vallejo']
fp = 'topics/lda_20_45_topics.csv'
df = pd.read_csv(fp)

In [ ]:
eval_cat(df, 20, 'PaloAlto')

### Load Best Model and Data

In [57]:
final_model = gensim.models.ldamodel.LdaModel.load('lda_data_c2000_3_60')
final_fp = 'topics/lda_data_c2000_3_60_2.csv'
final_model.show_topics(num_topics=60, formatted=False)

[(0,
  [('complaint', 0.1985935),
   ('log', 0.14190955),
   ('phone', 0.08581352),
   ('search', 0.083066285),
   ('warrant', 0.03568498),
   ('estate', 0.032001145),
   ('involved', 0.026251603),
   ('quality', 0.02599625),
   ('real', 0.025753625),
   ('ensure', 0.019929796)]),
 (1,
  [('associate', 0.14829391),
   ('client', 0.11122169),
   ('photograph', 0.10852045),
   ('office', 0.08708323),
   ('follow', 0.061714146),
   ('emergency', 0.042823374),
   ('transfer', 0.033178225),
   ('color', 0.024401158),
   ('restaurant', 0.02243146),
   ('time', 0.018097397)]),
 (2,
  [('name', 0.15037733),
   ('victim', 0.084349126),
   ('crime', 0.082644366),
   ('officer', 0.07667519),
   ('domestic', 0.041578013),
   ('violence', 0.03346935),
   ('injury', 0.030954132),
   ('capitol', 0.030555379),
   ('location', 0.027925178),
   ('type', 0.021750417)]),
 (3,
  [('permit', 0.2758547),
   ('issue', 0.11701758),
   ('building', 0.07663683),
   ('build', 0.06414055),
   ('building_permit', 0

In [74]:
data_type_dict = {0: 'Complaints to City',
                  1: 'Police Incident Report',
                  2: 'Police Incident Report',
                  3: 'Parcel Records, Permits, Plans',
                  4: '911/Law Enforcement Service Calls',
                  5: 'Public Works, Utilities',
                  6: 'Parcel Records, Permits, Plans',
                  7: 'Property Liens',
                  8: 'Purchasing Records, Contracts',
                  9: 'Criminal Record Check',
                  10: 'Criminal Record Check',
                  11: 'Purchasing Records, Contracts',
                  12: 'Criminal Record Check',
                  13: '911/Law Enforcement Service Calls',
                  14: 'City Government Meeting Notes',
                  15: 'Checks and Deposits',
                  16: 'Auto Collision Report',
                  17: 'Parcel Records, Permits, Plans',
                  18: 'Crime Photo and Video',
                  19: 'Police Incident Report',
                  20: 'Employee Benefits, Payroll',
                  21: 'Criminal Record Check',
                  22: 'Auto Collision Report',
                  23: 'Human Services Cases',
                  24: 'Environmental Assessment, Hazardous Materials',
                  25: 'Auto Collision Report',
                  26: 'City Emails, Social Media',
                  27: 'Uncategorized',
                  28: 'Police Incident Report',
                  29: 'Property Liens',
                  30: 'Building Code Violations',
                  31: 'Crime Photo and Video',
                  32: 'Uncategorized',
                  33: 'Auto Collision Report',
                  34: 'Police Incident Report',
                  35: 'City Emails, Social Media',
                  36: 'Police Incident Report',
                  37: 'Criminal Record Check',
                  38: 'Property Liens',
                  39: 'Employee Benefits, Payroll',
                  40: 'Crime Photo and Video',
                  41: 'Auto Collision Report',
                  42: 'Uncategorized',
                  43: 'Parcel Records, Permits, Plans',
                  44: 'Parcel Records, Permits, Plans',
                  45: 'Criminal Record Check',
                  46: 'Parcel Records, Permits, Plans',
                  47: 'Uncategorized',
                  48: 'Police Incident Report',
                  49: 'Criminal Record Check',
                  50: 'Uncategorized',
                  51: 'Public Works, Utilities',
                  52: 'Uncategorized',
                  53: 'Police Incident Report',
                  54: 'Uncategorized',
                  55: 'Environmental Assessment, Hazardous Materials',
                  56: 'Police Incident Report',
                  57: 'Public Works, Utilities',
                  58: 'Witness Statements',
                  59: 'Police Incident Report',
                  60: 'Police Administrative'}

In [75]:
final_data = highest_topic(final_fp)
final_data['data_type'] = final_data['top_topic'].apply(lambda x: data_type_dict[x])

In [136]:
final_data.to_csv('final_data.csv')

## Identify Most Popular Catgories

### "Winner take all" popularity metric:
* Scoring Rules:
    * Only the topic that composes the largest share of a document scores "points" for its "Adjusted Popularity" total.
    * If a topic composes the largest share of that document, its "points" are its composition score.


In [84]:
def winner_take_all(df, count, measure): 
    # categorize data by final model identify highest topics
    final_df = df[[measure, 'top_topic_comp']]
    if count:
        topic_gp = final_df.groupby(measure).count()
    else:
        topic_gp = final_df.groupby(measure).sum()
    topic_gp.reset_index(inplace = True)
    topic_gp.rename(index=str, columns={"top_topic_comp": "total_pop"}, inplace = True)
    topic_gp = topic_gp[[measure, 'total_pop']]

    if measure == 'top_topic':
        topic_gp.sort_values(by=['top_topic'], ascending = True)
        # add in topic words
        words_in_topics = [tup[1] for tup in final_model.show_topics(num_topics=60, formatted=False)] # update w/ winning model
        topic_gp['topic'] = words_in_topics

        topic_gp[["topic1", "topic2", "topic3", "topic4", 
               "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = topic_gp.topic.apply(pd.Series)

    topic_gp = topic_gp.sort_values(by = 'total_pop', ascending=False)
    topic_gp.reset_index(drop = True, inplace = True)
    return topic_gp

In [85]:
topic_gp_count = winner_take_all(final_data, True, 'top_topic')
topic_gp_sum = winner_take_all(final_data, False, 'top_topic')

In [65]:
topic_gp_count.to_csv('topics/final_model_wta_count.csv')
topic_gp_sum.to_csv('topics/final_model_wta_sum.csv')

In [87]:
wta_data_sum = winner_take_all(final_data, False, 'data_type')
wta_data_count = winner_take_all(final_data, True, 'data_type')

In [89]:
wta_data_sum.to_csv('topics/final_model_wta_count_data.csv')
wta_data_count.to_csv('topics/final_model_wta_sum_data.csv')

In [88]:
wta_data_sum

,data_type,total_pop
0,Police Incident Report,6056.105429
1,Auto Collision Report,5410.516739
2,"Parcel Records, Permits, Plans",3635.726240
3,Crime Photo and Video,3223.739908
4,Criminal Record Check,2514.669181
5,"Environmental Assessment, Hazardous Materials",1631.869153
6,"City Emails, Social Media",1352.116572
7,911/Law Enforcement Service Calls,1253.521343
8,Building Code Violations,1146.562057
9,Property Liens,968.321523


## "Winner Take All with Thresholds" Rules:

Scoring Rules:
* Same as "Winner Take All", except a winning topic must compose at least a certain threshold of a document to get any points.
* We'll try 0.2 (low) and 0.5 (high) thresholds.

In [95]:
def winner_thresh(df, thresh, count, measure):
    final_df = df[[measure, 'top_topic_comp']]


    final_df = final_df[final_df['top_topic_comp'] >= thresh]
    if count:
        topic_gp = final_df.groupby(measure).count()
    else:
        topic_gp = final_df.groupby(measure).sum()
        
    topic_gp.reset_index(inplace = True)
    topic_gp.rename(index=str, columns={"top_topic_comp": "total_pop"}, inplace = True)
    topic_gp = topic_gp[[measure, 'total_pop']]

    if measure == 'top_topic':
        topic_gp.sort_values(by=['top_topic'], ascending = True)

        # add in topic words
        words_in_topics = [tup[1] for tup in final_model.show_topics(num_topics=60, formatted=False)] # update w/ winning model
        topic_gp['topic'] = words_in_topics

        topic_gp[["topic1", "topic2", "topic3", "topic4", 
           "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = topic_gp.topic.apply(pd.Series)

    topic_gp = topic_gp.sort_values(by='total_pop', ascending=False)
    topic_gp.reset_index(drop = True, inplace = True)
    return topic_gp

In [97]:
topic2 = winner_thresh(final_data, 0.2, False, 'top_topic')
topic5 = winner_thresh(final_data, 0.5, False, 'top_topic')

In [96]:
topic2_dt = winner_thresh(final_data, 0.2, False, 'data_type')
topic5_dt = winner_thresh(final_data, 0.5, False, 'data_type')

In [98]:
topic2_dt

,data_type,total_pop
13,Police Incident Report,5826.818697
1,Auto Collision Report,5329.277310
12,"Parcel Records, Permits, Plans",3359.758105
7,Crime Photo and Video,3175.558612
8,Criminal Record Check,2371.972375
10,"Environmental Assessment, Hazardous Materials",1584.073551
4,"City Emails, Social Media",1221.631425
0,911/Law Enforcement Service Calls,1184.330523
2,Building Code Violations,1118.106614
16,"Purchasing Records, Contracts",886.408014


## Partial Credit Approach

* All topics assigned to a given PRR get credit for that PRR's topic composition score, provided the score is above the established threshold

In [114]:
def prop_calc(df, thresh, measure, data_type_dict = data_type_dict):
    results_dict = {}
    #df['topic_comp'] =  df['topic_comp'].apply(lambda x:  ast.literal_eval(x))
    for row_num in df.index:
        for tup in df.topic_comp[row_num]: 
            if tup[1] >= thresh: 
                if measure == 'top_topic':
                    if not tup[0] in results_dict:
                        results_dict[tup[0]] = tup[1] 
                    else:
                         results_dict[tup[0]] += tup[1]
                else:
                    if not data_type_dict[tup[0]] in results_dict:
                        results_dict[data_type_dict[tup[0]]] = tup[1] 
                    else:
                         results_dict[data_type_dict[tup[0]]] += tup[1]
            else:
                pass
                        
    pd_df = pd.DataFrame.from_dict(results_dict, orient = 'index')
    pd_df.reset_index(inplace = True)
    pd_df.rename(index = str, columns = {'index': measure, 0: 'total_score'}, inplace = True)
    #pd_df.topic = pd.to_numeric(pd_df.topic)
    pd_df = pd_df.sort_values(by=[measure], ascending = True)

    if measure == 'top_topic':
        # add in topic words
        words_in_topics = [tup[1] for tup in final_model.show_topics(num_topics=60, formatted=False)] # update w/ winning model
        pd_df['topic_words'] = words_in_topics

        pd_df[["topic1", "topic2", "topic3", "topic4", 
               "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = pd_df.topic_words.apply(pd.Series)

    
    
    pd_df = pd_df.sort_values(by='total_score', ascending=False)
    pd_df.reset_index(drop = True, inplace = True)
    return pd_df

In [108]:
prop_calc_2 = prop_calc(final_data, .2, 'top_topic')
prop_calc_5 = prop_calc(final_data, .5, 'top_topic')

In [115]:
prop_calc_2_dt = prop_calc(final_data, .2, 'data_type')
prop_calc_5_dt = prop_calc(final_data, .5, 'data_type')

In [116]:
prop_calc_2_dt

,data_type,total_score
0,Police Incident Report,7843.605297
1,Auto Collision Report,6220.267259
2,"Parcel Records, Permits, Plans",4488.424044
3,Crime Photo and Video,3767.362125
4,Criminal Record Check,3201.082004
5,"Environmental Assessment, Hazardous Materials",1882.571488
6,911/Law Enforcement Service Calls,1577.636194
7,"City Emails, Social Media",1525.486415
8,Property Liens,1424.262950
9,Building Code Violations,1371.284014


## Normalize Metrics within a City/County (Dampened Popularity):
* For each city/county, we add up total score fore each topic and then take the log of the total score. We then add up scores across each city/count.
* For winner-take-all, only score for top topic included (provided it is above threshold)
* For partial-credit, scores for all topics included (provided it is above threhsold
* This is an extra control for cities with a large number of PRRs from skewing our results


In [117]:
city_list = ['Arlington', 'Asheville', 'Bainbridge', 'Boulder', 'CathedralCity' ,'Clearwater', 'Dayton', 
            'Denton', 'Everett', 'FortCollins', 'Greensboro', 'Hayward', 'Kirkland', 'LasCruces', 'Lynnwood',
            'Mercer', 'Miami', 'Middleborough', 'Nola', 'Oakland', 'OKC', 'Olympia', 'PaloAlto', 
            'Peoria', 'Pullman', 'RanchoCucamonga', 'Redmond', 'Renton', 'Sacramento', 'SanFrancisco', 
            'Tukwila', 'Vallejo', 'WestSacramento', 'Winchester']

In [120]:
def norm_pop(df, city_list, thresh, winner_take_all, policy):
    #df = pd.read_csv(fp)
    #df['topic_comp'] =  df['topic_comp'].apply(lambda x:  ast.literal_eval(x))
    
    if policy:
        list_dicts_1 = []
        list_dicts_0 = []
        pd1 = {}
        pd0 = {}
    else:
        list_of_domain_dicts = []
        popularity_dict = {}
    
    for city in city_list:
        if policy:
            results_dict_1 = {}
            results_dict_0 = {}
        else:
            results_dict = {}
        
        #get our df only of rows from a given city/state domain
        city_df = df[df.city == city]
        
        for row_num in city_df.index:
            tup_list = city_df.topic_comp[row_num] #list of (topic, doc composition) tuples
            
            if winner_take_all:
        
                #return only the tuple w/highest topic composition value
                winner_tuple = max(tup_list, key=lambda item:item[1]) 
                if winner_tuple[1] > thresh:
                    if policy:
                        if city_df.policy[row_num] == 1:
                            if not winner_tuple[0] in results_dict_1: #if not in dict, add it with its TOTAL VIEWS score
                                results_dict_1[winner_tuple[0]] = winner_tuple[1] 
                            else:
                                results_dict_1[winner_tuple[0]] += winner_tuple[1]
                                
                        else:
                            if not winner_tuple[0] in results_dict_0: #if not in dict, add it with its TOTAL VIEWS score
                                results_dict_0[winner_tuple[0]] = winner_tuple[1] 
                            else:
                                results_dict_0[winner_tuple[0]] += winner_tuple[1]
                        
                        
                    else:
                        if not winner_tuple[0] in results_dict: #if not in dict, add it with its TOTAL VIEWS score
                            results_dict[winner_tuple[0]] = winner_tuple[1] 
                        else:
                            results_dict[winner_tuple[0]] += winner_tuple[1] 
                            
            else:
                if policy:
                    if city_df.policy[row_num] == 1:
                        for tup in tup_list: 
                            if tup[1] >= thresh:
                                if not tup[0] in results_dict_1:
                                    results_dict_1[tup[0]] = tup[1] 
                                else:
                                    results_dict_1[tup[0]] += tup[1] 
                    else:
                         for tup in tup_list: 
                            if tup[1] >= thresh:
                                if not tup[0] in results_dict_0:
                                    results_dict_0[tup[0]] = tup[1] 
                                else:
                                    results_dict_0[tup[0]] += tup[1]
                else:
                    
                    for tup in tup_list: 
                        if tup[1] >= thresh:
                            if not tup[0] in results_dict:
                                results_dict[tup[0]] = tup[1] 
                            else:
                                results_dict[tup[0]] += tup[1] 
            
        #when loop of domain_df is finished, take log of all keys in dict
        if policy:
            log_dict_1 = {}
            for k,v in results_dict_1.items():
                log_dict_1[k] = np.log(v) 
                
            log_dict_0 = {}
            for k,v in results_dict_0.items():
                log_dict_0[k] = np.log(v) 
        else:
            log_dict = {}
            for k,v in results_dict.items():
                log_dict[k] = np.log(v)

        if policy:
            list_dicts_1.append(log_dict_1)
            list_dicts_0.append(log_dict_0)
        else:
            #now we have a polished dict of topic numbers as keys and log of all views/DLs as values; append it to list
            list_of_domain_dicts.append(log_dict)
    
    #use Counter() object to sync our dictionaries
    if policy:
        c1 = Counter()
        for d1 in list_dicts_1:
            c1.update(d1)
        c0 = Counter()
        for d0 in list_dicts_0:
            c0.update(d0)
    else:  
        c = Counter()
        for d in list_of_domain_dicts:
            c.update(d)

    if policy:
        pd1 = dict(c1)
        pd0 = dict(c0)
    else:
        popularity_dict = dict(c)
    
    if policy:
        pd_df1 = pd.DataFrame.from_dict(pd1, orient = 'index')
        pd_df1.reset_index(inplace = True)
        pd_df1['policy'] = 1
        pd_df1.rename(index = str, columns = {'index': 'topic', 0: 'total_score'}, inplace = True)
        
        pd_df1.topic = pd.to_numeric(pd_df1.topic)
        pd_df1 = pd_df1.sort_values(by=['topic'], ascending = True)

        # add in topic words
        words_in_topics = [tup[1] for tup in final_model.show_topics(num_topics=60, formatted=False)] # update w/ winning model
        pd_df1['topic_words'] = words_in_topics

        pd_df1[["topic1", "topic2", "topic3", "topic4", 
               "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = pd_df1.topic_words.apply(pd.Series)
        
        pd_df0 = pd.DataFrame.from_dict(pd0, orient = 'index')
        pd_df0.reset_index(inplace = True)
        pd_df0['policy'] = 0
        pd_df0.rename(index = str, columns = {'index': 'topic', 0: 'total_score'}, inplace = True)
        pd_df0.topic = pd.to_numeric(pd_df0.topic)
        pd_df0 = pd_df0.sort_values(by=['topic'], ascending = True)

        # add in topic words
        words_in_topics = [tup[1] for tup in final_model.show_topics(num_topics=60, formatted=False)] # update w/ winning model
        pd_df0['topic_words'] = words_in_topics

        pd_df0[["topic1", "topic2", "topic3", "topic4", 
               "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = pd_df0.topic_words.apply(pd.Series)
    
        
        pd_df = pd.concat([pd_df0, pd_df1])
    else:
        pd_df = pd.DataFrame.from_dict(popularity_dict, orient = 'index')
        pd_df.reset_index(inplace = True)
    
        pd_df.rename(index = str, columns = {'index': 'topic', 0: 'total_score'}, inplace = True)
        pd_df.topic = pd.to_numeric(pd_df.topic)
        pd_df = pd_df.sort_values(by=['topic'], ascending = True)

        # add in topic words
        words_in_topics = [tup[1] for tup in final_model.show_topics(num_topics=60, formatted=False)] # update w/ winning model
        pd_df['topic_words'] = words_in_topics

        pd_df[["topic1", "topic2", "topic3", "topic4", 
               "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = pd_df.topic_words.apply(pd.Series)



    pd_df = pd_df.sort_values(by='total_score', ascending=False)

    return pd_df

In [121]:
np_df = norm_pop(pol_df, city_list, .2, True, True)

In [122]:
np_df[:100]

,topic,total_score,policy,topic_words,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10
39,55,75.800736,0,"[(environmental, 0.048749115), (hazardous, 0.0456608), (storage, 0.043166153), (material, 0.042970687), (hazardous_material, 0.034293775), (tank, 0.034046005), (property, 0.033831228), (site, 0.03341891), (assessment, 0.028234059), (storage_tank, 0.025999954)]","(environmental, 0.048749115)","(hazardous, 0.0456608)","(storage, 0.043166153)","(material, 0.042970687)","(hazardous_material, 0.034293775)","(tank, 0.034046005)","(property, 0.033831228)","(site, 0.03341891)","(assessment, 0.028234059)","(storage_tank, 0.025999954)"
45,30,70.835173,0,"[(violation, 0.12519231), (code, 0.08656456), (property, 0.06848214), (open, 0.06713956), (code_violation, 0.057894886), (zone, 0.04222084), (building, 0.03874377), (fire, 0.036294296), (unit, 0.030973408), (apn, 0.030083403)]","(violation, 0.12519231)","(code, 0.08656456)","(property, 0.06848214)","(open, 0.06713956)","(code_violation, 0.057894886)","(zone, 0.04222084)","(building, 0.03874377)","(fire, 0.036294296)","(unit, 0.030973408)","(apn, 0.030083403)"
16,26,66.901031,0,"[(email, 0.05471243), (send, 0.044262353), (letter, 0.0327627), (fee, 0.025692467), (disclosure, 0.025541063), (cost, 0.02162389), (mail, 0.021287313), (following, 0.020823436), (electronic, 0.01912893), (time, 0.01801677)]","(email, 0.05471243)","(send, 0.044262353)","(letter, 0.0327627)","(fee, 0.025692467)","(disclosure, 0.025541063)","(cost, 0.02162389)","(mail, 0.021287313)","(following, 0.020823436)","(electronic, 0.01912893)","(time, 0.01801677)"
13,3,59.168004,0,"[(permit, 0.2758547), (issue, 0.11701758), (building, 0.07663683), (build, 0.06414055), (building_permit, 0.059510738), (ref, 0.056181964), (construction, 0.03771535), (without, 0.024624562), (administration, 0.020187119), (since, 0.02015884)]","(permit, 0.2758547)","(issue, 0.11701758)","(building, 0.07663683)","(build, 0.06414055)","(building_permit, 0.059510738)","(ref, 0.056181964)","(construction, 0.03771535)","(without, 0.024624562)","(administration, 0.020187119)","(since, 0.02015884)"
2,46,58.777173,0,"[(locate, 0.12590905), (building, 0.11562841), (property, 0.074399255), (inspection, 0.07156111), (property_locate, 0.060741402), (plan, 0.059727844), (parcel, 0.052651398), (drawing, 0.0375317), (permit, 0.029983826), (spill, 0.028118191)]","(locate, 0.12590905)","(building, 0.11562841)","(property, 0.074399255)","(inspection, 0.07156111)","(property_locate, 0.060741402)","(plan, 0.059727844)","(parcel, 0.052651398)","(drawing, 0.0375317)","(permit, 0.029983826)","(spill, 0.028118191)"
1,43,57.687675,0,"[(project, 0.076902196), (plan, 0.029102111), (management, 0.02659861), (development, 0.025086623), (section, 0.024244718), (construction, 0.023521416), (work, 0.020521961), (improvement, 0.01656264), (electrical, 0.015781369), (compliance, 0.015751444)]","(project, 0.076902196)","(plan, 0.029102111)","(management, 0.02659861)","(development, 0.025086623)","(section, 0.024244718)","(construction, 0.023521416)","(work, 0.020521961)","(improvement, 0.01656264)","(electrical, 0.015781369)","(compliance, 0.015751444)"
12,44,57.169770,0,"[(plan, 0.11660539), (site, 0.099882215), (certificate, 0.09434634), (occupancy, 0.08399464), (certificate_occupancy, 0.07622669), (site_plan, 0.045080233), (final, 0.040189173), (permit, 0.034261044), (interested, 0.03197086), (variance, 0.025503768)]","(plan, 0.11660539)","(site, 0.099882215)","(certificate, 0.09434634)","(occupancy, 0.08399464)","(certificate_occupancy, 0.07622669)","(site_plan, 0.045080233)","(final, 0.040189173)","(permit, 0.034261044)","(interested, 0.03197086)","(variance, 0.025503768)"
40,11,53.875991,0,"[(contract, 0.12024548), (bid, 0.08086701), (submit, 0.0620963), (ordinance, 0.048248757), (subcontractor, 0.04418181), (service, 0.042105265), (project, 0.041909378), (rfp, 0.036090374), (approve, 0.03256977), (award, 0.031801503)]","(contract, 0.12024548)","(b

In [127]:
def np_topic_to_data(np_df, data_type_dict = data_type_dict):
    np_df['data_type'] = np_df['topic'].apply(lambda x: data_type_dict[x])
    np_gp = np_df.groupby(['data_type', 'policy']).sum()
    np_gp.reset_index(inplace= True)
    np_gp.sort_values(by = 'data_type', inplace = True)
    np_gp.reset_index(drop = True, inplace = True)
    return np_gp

In [128]:
np_data_type = np_topic_to_data(np_df)

In [129]:
np_data_type

,data_type,policy,topic,total_score
0,911/Law Enforcement Service Calls,0,17,57.869420
1,911/Law Enforcement Service Calls,1,17,18.706199
2,Auto Collision Report,0,137,141.158600
3,Auto Collision Report,1,137,20.397709
4,Building Code Violations,0,30,70.835173
5,Building Code Violations,1,30,13.247308
6,Checks and Deposits,0,15,17.670792
7,Checks and Deposits,1,15,6.212712
8,"City Emails, Social Media",1,61,15.497099
9,"City Emails, Social Media",0,61,83.122713


In [159]:
np_df = norm_pop(final_fp, city_list, .2, False)

In [56]:
np_df.to_csv('topics/np_pop_c2000_60_pol.csv')

In [ ]:
norm_pop('topics/lda_data_avg.csv', city_list, .5, False)

In [ ]:
norm_pop('topics/lda_data_avg.csv', city_list, .2, False)

## Topic Popularity by City

In [131]:
def pop_by_city(fp, city_list, num_topics, thresh, winner_take_all):
    
    df = pd.read_csv(fp)
    df['topic_comp'] =  df['topic_comp'].apply(lambda x:  ast.literal_eval(x))
    
    cols = ['city', 'topic', 'total_pop', 'pct'] 
    
    topic_pop_city = pd.DataFrame(columns = cols)
    
    for city in city_list:
        
        results_dict = {}
        for i in range(0, num_topics):
            results_dict[i] = 0
        
        #get our df only of rows from a given city/state domain
        city_df = df[df.city == city]
        
        for row_num in city_df.index:
            tup_list = city_df.topic_comp[row_num] #list of (topic, doc composition) tuples
            
            if winner_take_all:
        
                #return only the tuple w/highest topic composition value
                winner_tuple = max(tup_list, key=lambda item:item[1])  

                if not winner_tuple[0] in results_dict: #if not in dict, add it with its TOTAL VIEWS score
                    if winner_tuple[1] > thresh:
                        results_dict[winner_tuple[0]] = winner_tuple[1] 
                else:
                    pass

                if winner_tuple[0] in results_dict: #if in dict, increment that key's value with score
                    if winner_tuple[1] > thresh:
                        results_dict[winner_tuple[0]] += winner_tuple[1]
                    pass
            else:
                for tup in tup_list: 
                    if not tup[0] in results_dict:
                        if tup[1] >= thresh: 
                            results_dict[tup[0]] = tup[1] 
                    else:
                        pass
                    if tup[0] in results_dict:
                        if tup[1] >= thresh:
                            results_dict[tup[0]] += tup[1] 
        
        pd_df = pd.DataFrame.from_dict(results_dict, orient = 'index')
        pd_df.reset_index(inplace = True)
        pd_df.rename(index = str, columns = {'index': 'topic', 0: 'total_pop'}, inplace = True)
        pd_df['city'] = city
        pd_df['pct'] = (pd_df['total_pop']/sum(pd_df['total_pop']))*100
        topic_pop_city = pd.concat([topic_pop_city, pd_df])

    return topic_pop_city

In [132]:
pbc = pop_by_city(final_fp, city_list, 60, .2, False)

In [134]:
pbc['data_type'] = pbc['topic'].apply(lambda x: data_type_dict[x])

In [135]:
pbc.to_csv('pbc.csv')

## City Deep-Dive

In [2]:
clean_data = pd.read_csv('../data/clean_data.csv')

In [5]:
clean_data.head()

,month_year,count,count_req,city_x,city_x.1,state_x,months,policy_months,portal_months,policy,portal,population,median_age,median_gross_rent,median_income,pct_25_34,pct_male,pct_black,pct_white,pct_bachelor,Budget_1617,2015,2016,2017,2018,mayor_council,council_manager,dem,rep,population_09,median_age_09,median_income_09,pct_25_34_09,pct_male_09,pct_black_09,pct_white_09,pct_bachelor_09,robust_portal,treat_score,year,robust_policy,count_diff,req_ratio,policy_lag3,policy_lag6,portal_lag3,portal_lag6,policy_months_lag3,policy_months_lag6,portal_months_lag3,portal_months_lag6,policy_3_6mos,policy_6_12mos,policy_1_2year,policy_2_3year,policy_over3year,portal_3_6mos,portal_6_12mos,portal_1_2year,portal_2_3year,portal_over3year,time_complete,treatment_pol,treatment_por,city
0,2015-08-01,32,24.0,Albuquerque city,Albuquerque city,NM,7,0.0,43.0,0,1,556859,36.0,816,26910.0,0.157106,0.486782,0.032753,0.720615,0.125156,526400000.0,1,0,0,0,1,0,1,0,515107,34.3,25584.0,0.164783,0.490131,0.034793,0.682959,0.119395,1,2,2015,0,8.0,0.750000,0,0,1,1,0,0,40,37,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,143.050152,0.0,1.0,Albuquerque
1,2015-09-01,149,102.0,Albuquerque city,Albuquerque city,NM,8,0.0,44.0,0,1,556859,36.0,816,26910.0,0.157106,0.486782,0.032753,0.720615,0.125156,526400000.0,1,0,0,0,1,0,1,0,515107,34.3,25584.0,0.164783,0.490131,0.034793,0.682959,0.119395,1,2,2015,0,47.0,0.684564,0,0,1,1,0,0,41,38,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,145.507056,0.0,1.0,Albuquerque
2,2015-10-01,155,118.0,Albuquerque city,Albuquerque city,NM,9,0.0,45.0,0,1,556859,36.0,816,26910.0,0.157106,0.486782,0.032753,0.720615,0.125156,526400000.0,1,0,0,0,1,0,1,0,515107,34.3,25584.0,0.164783,0.490131,0.034793,0.682959,0.119395,1,2,2015,0,37.0,0.761290,0,0,1,1,0,0,42,39,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,119.474763,0.0,1.0,Albuquerque
3,2015-11-01,226,163.0,Albuquerque city,Albuquerque city,NM,10,0.0,46.0,0,1,556859,36.0,816,26910.0,0.157106,0.486782,0.032753,0.720615,0.125156,526400000.0,1,0,0,0,1,0,1,0,515107,34.3,25584.0,0.164783,0.490131,0.034793,0.682959,0.119395,1,2,2015,0,63.0,0.721239,0,0,1,1,0,0,43,40,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,77.379855,0.0,1.0,Albuquerque
4,2015-12-01,234,158.0,Albuquerque city,Albuquerque city,NM,11,0.0,47.0,0,1,556859,36.0,816,26910.0,0.157106,0.486782,0.032753,0.720615,0.125156,526400000.0,1,0,0,0,1,0,1,0,515107,34.3,25584.0,0.164783,0.490131,0.034793,0.682959,0.119395,1,2,2015,0,76.0,0.675214,0,0,1,1,0,0,44,41,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,43.056918,0.0,1.0,Albuquerque


In [4]:
cs = clean_data['city_x'].str.split(" ", expand = True)
clean_data['city'] = cs[0]

In [6]:
clean_data['my_city'] = clean_data['month_year']+clean_data['city']

In [7]:
clean_data = clean_data[['my_city', 'policy', 'portal', 'robust_policy', 'robust_portal', 'policy_months', 'portal_months']]

In [20]:
final_df = highest_topic(final_fp)
final_df['month_year'] = final_df['month_year']+'-01'
final_df['my_city'] = final_df['month_year'] + final_df['city']

In [19]:
clean_data[:200]

,my_city,policy,portal,robust_policy,robust_portal,policy_months,portal_months
0,2015-08-01Albuquerque,0,1,0,1,0.0,43.0
1,2015-09-01Albuquerque,0,1,0,1,0.0,44.0
2,2015-10-01Albuquerque,0,1,0,1,0.0,45.0
3,2015-11-01Albuquerque,0,1,0,1,0.0,46.0
4,2015-12-01Albuquerque,0,1,0,1,0.0,47.0
5,2016-01-01Albuquerque,0,1,0,1,0.0,48.0
6,2016-02-01Albuquerque,0,1,0,1,0.0,49.0
7,2016-03-01Albuquerque,0,1,0,1,0.0,50.0
8,2016-04-01Albuquerque,0,1,0,1,0.0,51.0
9,2016-05-01Albuquerque,0,1,0,1,0.0,52.0


In [21]:
pol_df = final_df.merge(clean_data, how ='left', on = 'my_city')

In [23]:
pol_df = pol_df[pol_df['month_year'] != '2018-06-01']

In [25]:
pol_df.to_csv('pol_df.csv')

In [17]:
city_dd = ['Clearwater','Greensboro', 'FortCollins', 'Nola']

for city in city_dd:
    city_df = final_df[final_df['city'] == city]
    month_count = city_df.groupby(['month_year', 'top_topic']).sum()
    month_count = month_count.reset_index()
    month_count = month_count[['month_year', 'top_topic', 'top_topic_comp']]
   
    fn = '{}_topic.csv'.format(city)
    month_count.to_csv(fn)
    
    df = month_count.groupby('top_topic').sum()
    df.sort_values(by = 'top_topic_comp', ascending = False, inplace = True)
    
    fn_tt = '{}_top_topic.csv'.format(city)
    df.to_csv(fn_tt)

In [18]:
final_df.head()

,Unnamed: 0,index,Summary,city,month_year,token,lemma,mash,mash_len,bigrams,common_bigrams,final_mash,count_total,topic_comp,comp_len,top_topic,top_topic_comp
0,0,0,We are working with an engineering firm on an upcoming project They have asked us to gather maps for this project Would you be able to assist me in gathering maps records as builds for any underground water facilities you may have Something just showing the route of the water lines would do\n\nth ST NE to nd Ave NE Arlington Cascade Surveying Engineering \n\nI have attached the scope for your convenience Please let me know if you have questions,Arlington,2018-06,"['are', 'working', 'with', 'an', 'engineering', 'firm', 'on', 'an', 'upcoming', 'project', 'they', 'have', 'asked', 'us', 'to', 'gather', 'maps', 'for', 'this', 'project', 'would', 'you', 'be', 'able', 'to', 'assist', 'me', 'in', 'gathering', 'maps', 'records', 'as', 'builds', 'for', 'any', 'underground', 'water', 'facilities', 'you', 'may', 'have', 'something', 'just', 'showing', 'the', 'route', 'of', 'the', 'water', 'lines', 'would', 'do', 'th', 'st', 'ne', 'to', 'nd', 'ne', 'cascade', 'surveying', 'engineering', 'i', 'have', 'attached', 'the', 'scope', 'for', 'your', 'convenience', 'please', 'let', 'me', 'know', 'if', 'you', 'have', 'questions']","[('are', 'VBP'), ('working', 'VBG'), ('with', 'IN'), ('an', 'DT'), ('engineering', 'NN'), ('firm', 'NN'), ('on', 'IN'), ('an', 'DT'), ('upcoming', 'JJ'), ('project', 'NN'), ('they', 'PRP'), ('have', 'VBP'), ('asked', 'VBN'), ('us', 'PRP'), ('to', 'TO'), ('gather', 'VB'), ('maps', 'NNS'), ('for', 'IN'), ('this', 'DT'), ('project', 'NN'), ('would', 'MD'), ('you', 'PRP'), ('be', 'VB'), ('able', 'JJ'), ('to', 'TO'), ('assist', 'VB'), ('me', 'PRP'), ('in', 'IN'), ('gathering', 'VBG'), ('maps', 'NNS'), ('records', 'NNS'), ('as', 'IN'), ('builds', 'NNS'), ('for', 'IN'), ('any', 'DT'), ('underground', 'JJ'), ('water', 'NN'), ('facilities', 'NNS'), ('you', 'PRP'), ('may', 'MD'), ('have', 'VB'), ('something', 'NN'), ('just', 'RB'), ('showing', 'VBG'), ('the', 'DT'), ('route', 'NN'), ('of', 'IN'), ('the', 'DT'), ('water', 'NN'), ('lines', 'NNS'), ('would', 'MD'), ('do', 'VB'), ('th', 'VB'), ('st', 'VB'), ('ne', 'JJ'), ('to', 'TO'), ('nd', 'VB'), ('ne', 'JJ'), ('cascade', 'NN'), ('surveying', ...","['work', 'engineering', 'firm', 'upcoming', 'project', 'gather', 'map', 'project', 'assist', 'gather', 'map', 'build', 'underground', 'water', 'facility', 'something', 'show', 'route', 'water', 'line', 'cascade', 'survey', 'engineering', 'scope', 'convenience']",25,"['work_engineering', 'engineering_firm', 'firm_upcoming', 'upcoming_project', 'project_gather', 'gather_map', 'map_project', 'project_would', 'would_assist', 'assist_gather', 'gather_map', 'map_record', 'record_build', 'build_underground', 'underground_water', 'water_facility', 'facility_something', 'something_show', 'show_route', 'route_water', 'water_line', 'line_would', 'would_cascade', 'cascade_survey', 'survey_engineering', 'engineering_attach', 'attach_scope', 'scope_convenience', 'convenience_please', 'please_know']",[],"['work', 'engineering', 'firm', 'upcoming', 'project', 'gather', 'map', 'project', 'assist', 'gather', 'map', 'build', 'underground', 'water', 'facility', 'something', 'show', 'route', 'water', 'line', 'cascade', 'survey', 'engineering', 'scope', 'convenience']",28604,"[(3, 0.039502017), (5, 0.11602564), (15, 0.03910256), (17, 0.11602564), (26, 0.041392785), (30, 0.03910256), (39, 0.13302885), (43, 0.24310635), (47, 0.038521882), (55, 0.050848946), (56, 0.03910256), (57, 0.07347096)]",12,43,0.243106
1,1,1,Need copies of contracts and all related documents pertaining to Topcub Aircraft property located at th DR NE Arlington WA between Arlington Airport Topcub Aircraft City of Arlington HCI Steel Buildings and PUD,Arlington,2018-06,"['copies', 'of', 'contracts', 'and', 'all', 'related', 'documents', 'pertaining', 'to', 'topcub', 'aircraft', 'property', 'located', 'at', 'th', 'dr', 'ne', 'wa', '

## Topic Popularity Over Time

In [62]:
final_df.head()

,Unnamed: 0,index,Summary,city,month_year,token_sp,pn,sum_ed,token,lemma,mash,mash_len,bigrams,common_bigrams,final_mash,ave,phrase,count_total,topic_comp
0,0,0,We are working with an engineering firm on an upcoming project They have asked us to gather maps for this project Would you be able to assist me in gathering mapsrecords as builds for any underground water facilities you may have Something just showing the route of the water lines would do\n\nth ST NE to nd Ave NE Arlington Cascade Surveying Engineering \n\nI have attached the scope for your convenience Please let me know if you have questions,Arlington,2018-06,We are working with an engineering firm on an upcoming project They have asked us to gather maps for this project Would you be able to assist me in gathering mapsrecords as builds for any underground water facilities you may have Something just showing the route of the water lines would do\n\nth ST NE to nd Ave NE Arlington Cascade Surveying Engineering \n\nI have attached the scope for your convenience Please let me know if you have questions,"['ave', 'ne', 'arlington', 'cascade', 'surveying', 'engineering']",we are working with an engineering firm on an upcoming project they have asked us to gather maps for this project would you be able to assist me in gathering mapsrecords as builds for any underground water facilities you may have something just showing the route of the water lines would do\n\nth st ne to nd ave ne arlington cascade surveying engineering \n\ni have attached the scope for your convenience please let me know if you have questions,"['are', 'working', 'with', 'an', 'engineering', 'firm', 'on', 'an', 'upcoming', 'project', 'they', 'have', 'asked', 'us', 'to', 'gather', 'maps', 'for', 'this', 'project', 'would', 'you', 'be', 'able', 'to', 'assist', 'me', 'in', 'gathering', 'mapsrecords', 'as', 'builds', 'for', 'any', 'underground', 'water', 'facilities', 'you', 'may', 'have', 'something', 'just', 'showing', 'the', 'route', 'of', 'the', 'water', 'lines', 'would', 'do', 'th', 'st', 'ne', 'to', 'nd', 'ne', 'cascade', 'surveying', 'engineering', 'i', 'have', 'attached', 'the', 'scope', 'for', 'your', 'convenience', 'please', 'let', 'me', 'know', 'if', 'you', 'have', 'questions']","[('are', 'VBP'), ('working', 'VBG'), ('with', 'IN'), ('an', 'DT'), ('engineering', 'NN'), ('firm', 'NN'), ('on', 'IN'), ('an', 'DT'), ('upcoming', 'JJ'), ('project', 'NN'), ('they', 'PRP'), ('have', 'VBP'), ('asked', 'VBN'), ('us', 'PRP'), ('to', 'TO'), ('gather', 'VB'), ('maps', 'NNS'), ('for', 'IN'), ('this', 'DT'), ('project', 'NN'), ('would', 'MD'), ('you', 'PRP'), ('be', 'VB'), ('able', 'JJ'), ('to', 'TO'), ('assist', 'VB'), ('me', 'PRP'), ('in', 'IN'), ('gathering', 'VBG'), ('mapsrecords', 'NNS'), ('as', 'IN'), ('builds', 'NNS'), ('for', 'IN'), ('any', 'DT'), ('underground', 'JJ'), ('water', 'NN'), ('facilities', 'NNS'), ('you', 'PRP'), ('may', 'MD'), ('have', 'VB'), ('something', 'NN'), ('just', 'RB'), ('showing', 'VBG'), ('the', 'DT'), ('route', 'NN'), ('of', 'IN'), ('the', 'DT'), ('water', 'NN'), ('lines', 'NNS'), ('would', 'MD'), ('do', 'VB'), ('th', 'VB'), ('st', 'VB'), ('ne', 'JJ'), ('to', 'TO'), ('nd', 'VB'), ('ne', 'JJ'), ('cascade', 'NN'), ('surveying', 'VBG'), ('eng...","['work', 'engineering', 'firm', 'upcoming', 'project', 'gather', 'map', 'project', 'assist', 'gather', 'mapsrecords', 'build', 'underground', 'water', 'facility', 'something', 'show', 'route', 'water', 'line', 'cascade', 'survey', 'engineering', 'scope', 'convenience']",41,"['work_engineering', 'engineering_firm', 'firm_upcoming', 'upcoming_project', 'project_gather', 'gather_map', 'map_project', 'project_assist', 'assist_gather', 'gather_mapsrecords', 'mapsrecords_build', 'build_underground', 'underground_water', 'water_facility', 'facility_something', 'something_show', 'show_route', 'route_water', 'water_line', 'line_cascade', 'cascade_survey', 'survey_engineering', 'engineering_scope', 'scope_convenience']",[],"['work', 'engineering', 'fi

In [ ]:
def topic popularity(city):
    avg_month.plot(x='date_posted', y = ['total_price_excluding_optional_support', 'total_price_including_optional_support'], kind = 'line' )
    plt.show()

## Scratch

In [ ]:
df_sp = pd.read_csv('topics/lda_data_sp.csv')

In [ ]:
df_sp['final_mash'] = df_sp['final_mash'].apply(lambda x:  ast.literal_eval(x))
df_sp['mash'] = df_sp['mash'].apply(lambda x:  ast.literal_eval(x))
df_sp['common_bigrams'] = df_sp['common_bigrams'].apply(lambda x:  ast.literal_eval(x))
df_sp['token'] = df_sp['token'].apply(lambda x:  ast.literal_eval(x))
df_sp['lemma'] = df_sp['lemma'].apply(lambda x:  ast.literal_eval(x))
df_sp['pn2'] = df_sp['pn2'].apply(lambda x:  ast.literal_eval(x))


In [ ]:
pn_list = [y for x in list(df_sp['pn2']) for y in x]
pn_counts = Counter(pn_list)
words = list(pn_counts.keys())
cnt = list(pn_counts.values())
pn_count_df = pd.DataFrame({'word': words, 'cnt': cnt})

In [ ]:
pn_count_df.cnt.describe()

In [ ]:
pn_count_df4 = pn_count_df[pn_count_df['cnt'] == 4]

In [ ]:
pn_count_df4[:100]

In [ ]:
pn_count_df.sort_values(by = ['cnt'], ascending = False, inplace = True)

In [ ]:
pn_count_df[1000:1100]